# CSC-6621: Final Project 

### Topic: Predicting Patient Readmission Rates using Random Forests and Gradient Boosting
**Team Members:** Sumedha Sanjeev, Ethan Traas, Ben Paulson<br/>
**Professor:** Dr. Berisha

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## Part 1: Getting and Analyzing the Data
As mentioned in our official proposal for exploring this domain, this data comes from [data.cms.gov](https://data.cms.gov/provider-data/dataset/9n3s-kdb3#data-table), formally named "The Center of Medicare and Medicaid". The following description is provided by CMS as part of the Hospital Readmissions Reduction Program:
```text
In October 2012, CMS began reducing Medicare payments for subsection(d) hospitals with excess readmissions under the Hospital Readmissions Reduction Program (HRRP). Excess readmissions are measured by a ratio, calculated by dividing a hospital's predicted rate of readmissions for heart attack (AMI), heart failure (HF), pneumonia, chronic obstructive pulmonary disease (COPD), hip/knee replacement (THA/TKA), and coronary artery bypass graft surgery (CABG) by the expected rate of readmissions, based on an average hospital with similar patients.
```

The data is provided in a CSV file, which we will download and explore in the following `Part 1` section. It is a public dataset, and this is the data description:
* **Released:** December 10, 2020
* **Last Updated:** July 31, 2024
* **Rows:** 18774
* **Features (12):**
    * Facility Name
    * Facility ID
    * State
    * Measure Name
    * Number of Discharges
    * Footnote
    * Excess Readmission Ratio
    * Predicted Readmission Rate
    * Expected Readmission Rate
    * Number of Readmissions
    * Start Date
    * End Date



In [2]:
data = pd.read_csv('raw-data/FY_2024_Hospital_Readmissions_Reduction_Program_Hospital.csv')
data

,Facility Name,Facility ID,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,NaN,NaN,0.8916,3.5325,3.9618,Too Few to Report,07/01/2019,06/30/2022
1,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HF-HRRP,616.0,NaN,1.1003,23.1263,21.0184,149,07/01/2019,06/30/2022
2,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-AMI-HRRP,274.0,NaN,0.9332,12.9044,13.8283,32,07/01/2019,06/30/2022
3,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-PN-HRRP,404.0,NaN,0.9871,17.0529,17.2762,68,07/01/2019,06/30/2022
4,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-CABG-HRRP,126.0,NaN,0.9517,9.8131,10.3112,11,07/01/2019,06/30/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
18769,TRINITY REGIONAL HOSPITAL SACHSE,670319,TX,READM-30-HIP-KNEE-HRRP,NaN,5.0,NaN,NaN,NaN,NaN,07/01/2019,06/30/2022
18770,TRINITY REGIONAL HOSPITAL SACHSE,670319,TX,READM-30-HF-HRRP,NaN,1.0,NaN,NaN,NaN,NaN,07/01/2019,06/30/2022
18771,TRINITY REGIONAL HOSPITAL SACHSE,670319,TX,READM-30-COPD-HRRP,NaN,5.0,NaN,NaN,NaN,NaN,07/01/2019,06/30/2022
18772,TRINITY REGIONAL HOSPITAL SACHSE,670319,TX,READM-30-CABG-HRRP,NaN,5.0,NaN,NaN,NaN,NaN,07/01/2019,06/30/2022


In [3]:
print(data.dtypes)

Facility Name                  object
Facility ID                     int64
State                          object
Measure Name                   object
Number of Discharges          float64
Footnote                      float64
Excess Readmission Ratio      float64
Predicted Readmission Rate    float64
Expected Readmission Rate     float64
Number of Readmissions         object
Start Date                     object
End Date                       object
dtype: object


In [4]:
print(data.describe())


         Facility ID  Number of Discharges     Footnote  \
count   18774.000000           8094.000000  6697.000000   
mean   261506.077661            244.251915     3.147230   
std    163743.437584            227.648660     2.084349   
min     10001.000000              0.000000     1.000000   
25%    110086.000000            105.000000     1.000000   
50%    250049.000000            176.000000     5.000000   
75%    390113.000000            308.000000     5.000000   
max    670319.000000           4994.000000     7.000000   

       Excess Readmission Ratio  Predicted Readmission Rate  \
count              12077.000000                12077.000000   
mean                   1.001776                   15.098235   
std                    0.077018                    5.480391   
min                    0.615900                    1.927900   
25%                    0.958000                   12.436800   
50%                    0.997800                   16.485500   
75%                    1.04

## Part 2: Preprocessing the Data
Because this data is supplied from a variety of sources across different hostpitals, it is not perfectly formatted yet in order to be used for machine learning. Therefore, we will need to preprocess the data before we can use it to train a model. The following steps will be taken to preprocess the data:
* General data cleaning
    * Remove leading "READM-30-" and trailing "-HRRP" from `Measure Name`
    * Make a new column `Length of Stay` from `Start Date` and `End Date`
* Handling missing/unhelpful values
    * Drop columns with too many missing values
    * Drop columns that are not useful (`Start Date`, `End Date` are all the same)
* Encode categorical variables
    * `Facility Name`, `State`, `Measure Name`
* Standardize numerical variables (0-1)
    * `Number of Discharges`, `Excess Readmission Ratio`, `Predicted Readmission Rate`, `Expected Readmission Rate`, `Number of Readmissions`
* Define the target variable
    * **Target:** `Predicted Readmission Rate`


#### Part 2.1: General Data Cleaning

In [5]:
# Cleaning the Measure Name column
data['Measure Name'] = data['Measure Name'].str.replace('READM-30-', '')
data['Measure Name'] = data['Measure Name'].str.replace('-HRRP', '')

# Determining the length of stay
data['Length of Stay'] = pd.to_datetime(data['End Date']) - pd.to_datetime(data['Start Date'])
data['Length of Stay'] = data['Length of Stay'].dt.days

#### Part 2.2: Remove Unhelpful Data
* `StartDate` and `EndDate` are the same, so we can drop one of them
* `Facility Name` is captured numerically by `Facility ID`

In [6]:
# Drop the `Start Date` and `End Date` and 'Facility Name' columns
data = data.drop(columns=['Start Date', 'End Date', 'Facility Name'])

#### Part 2.3: Encode Categorical Variables

In [7]:
# Encode categorical columns (Facility Name, State, Measure Name)
data = pd.get_dummies(data, columns=['State', 'Measure Name'])

#### Part 2.4: Standardize Numerical Variables

In [8]:
# Standardize numerical values; 0-1 (Number of Discharges, Excess Readmission Ratio, Predicted Readmission Rate, Expected Readmission Rate, Number of Readmissions)

# Convert any non-numeric values in above columns to NaN
data['Number of Discharges'] = pd.to_numeric(data['Number of Discharges'], errors='coerce')
data['Excess Readmission Ratio'] = pd.to_numeric(data['Excess Readmission Ratio'], errors='coerce')
data['Predicted Readmission Rate'] = pd.to_numeric(data['Predicted Readmission Rate'], errors='coerce')
data['Expected Readmission Rate'] = pd.to_numeric(data['Expected Readmission Rate'], errors='coerce')
data['Number of Readmissions'] = pd.to_numeric(data['Number of Readmissions'], errors='coerce')

# Standardize
data['Number of Discharges'] = (data['Number of Discharges'] - data['Number of Discharges'].min()) / (data['Number of Discharges'].max() - data['Number of Discharges'].min())
data['Excess Readmission Ratio'] = (data['Excess Readmission Ratio'] - data['Excess Readmission Ratio'].min()) / (data['Excess Readmission Ratio'].max() - data['Excess Readmission Ratio'].min())
data['Predicted Readmission Rate'] = (data['Predicted Readmission Rate'] - data['Predicted Readmission Rate'].min()) / (data['Predicted Readmission Rate'].max() - data['Predicted Readmission Rate'].min())
data['Expected Readmission Rate'] = (data['Expected Readmission Rate'] - data['Expected Readmission Rate'].min()) / (data['Expected Readmission Rate'].max() - data['Expected Readmission Rate'].min())
data['Number of Readmissions'] = (data['Number of Readmissions'] - data['Number of Readmissions'].min()) / (data['Number of Readmissions'].max() - data['Number of Readmissions'].min())

#### Part 2.5: Define the Target Variable

In [9]:
# Identify the target variable
data.rename(columns={'Predicted Readmission Rate': 'output'}, inplace=True)

# Drop rows where the target variable is NaN
data = data.dropna(subset=['output'])

#### Part 2.6: Determine What to do with NaN Values

`hit_list` is a list of the columns that have NaN values, as well as the percentage of that data which is NOT NaN
```text
('Number of Discharges', 65.33079407137534),
 ('Footnote', 0.0),
 ('Number of Readmissions', 65.33079407137534)]
 ```

 Depending on the model, we will either keep the NaN values or replace them with the average value of the column. For instance, one of the methods that will be tested (`XGBoost`) supports the interpretation of NaN.

In [10]:
# Identify which columns contain NaN values
hit_list = []
for column_name in list(data.columns):
    data_without_na = data.dropna(subset=[column_name])
    percentage = len(data_without_na) / len(data) * 100
    if percentage != 100:
        hit_list.append((column_name, percentage))

for hit in hit_list:
    if hit[1] < 50:
        data = data.drop(columns=[hit[0]])
    else:
        data[hit[0]] = data[hit[0]].fillna(data[hit[0]].mean())

#### Part 2.7: Get Data in State for SKLearn Models

In [11]:
# Format data for sklearn model training
data.to_csv('cleaned_data.csv', index=False)

## Part 2: Random Forest Predictions

The baseline model that we will be testing for this domain is the `Random Forest`. If you don't know what the Random Forest model is, you can read more about it [here](https://en.wikipedia.org/wiki/Random_forest). In summary, the Random Forest model is an ensemble learning method that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

**The following steps will be taken to train and test the Random Forest model:**
* Split the data into training and testing sets
* Train the Random Forest model
    * Use the `RandomForestRegressor` model from `sklearn`
    * Hyperparameter `n_estimators` will be tuned. This hyperparameter is responsible for the number of trees in the forest. The larger the number of trees, the better the model will be but the longer it will take to run. Another downside of increasing the number of trees is that the model may overfit.
    * Hyperparameter `max_depth` will be tuned. This hyperparameter is responsible for the maximum depth of the tree. The deeper the tree, the more splits it has and it captures more information about the data. However, the deeper the tree, the more likely it is to overfit.
* Results will be evaluated
    * Output is `MSE` since this is a regression model. `MSE` is the mean squared error, which is the average of the squares of the errors. The larger the number, the larger the error. A good value to shoot for, given our output is a percentage, is 0.01 or less.

A random forest model was chosen because it is a robust model that can handle a large number of features and is less likely to overfit than a single decision tree. It is also a good model to use for regression problems because it handles non-linear relationships by the combination of the linear nature of decision trees.


In [12]:
# Import random forest from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#### Part 2.1: Data Setup

In [13]:
data = pd.read_csv('cleaned_data.csv')

In [14]:
# Split the data into training and testing sets
X = data.drop(columns=['output'])
y = data['output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Part 2.2: Train a Random Forest Model

In [15]:
# Instantiate and train a Random Forest Model
model = RandomForestRegressor(n_estimators=100, max_depth=10)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10)

In [16]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(mse)

1.4723833011278562e-05


#### Part 2.3: Hyperparameter Tune via Grid Search
Given we achieved an MSE of `1.1657932133341314e-05`, we can say that the Random Forest model is a good fit for this data. The next step is to try optimizing this model to achieve better results while visualizing how the mse changes given different parameters. We will use a simple two-level for-loop to iterate through different values of `n_estimators` and `max_depth` to find the best combination of hyperparameters.

The range of values which will be tested are as follows:
```python
n_estimators = [10, 25, 50, 100, 150, 200, 300, 400]
max_depth = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 60]
```

In [17]:
n_estimators = [10, 25, 50, 100, 150, 200, 300, 400]
max_depth = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 60]

results = []

for n in tqdm(n_estimators):
    for d in max_depth:
        model = RandomForestRegressor(n_estimators=n, max_depth=d)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        results.append((n, d, mse))

100%|██████████| 8/8 [03:57<00:00, 29.69s/it]


In [18]:
# Determine the best hyperparameters
best = min(results, key=lambda x: x[2])
print(f"""
Best hyperparameters:
- n_estimators: {best[0]}
- max_depth: {best[1]}
- mse: {best[2]}
""")


Best hyperparameters:
- n_estimators: 300
- max_depth: 40
- mse: 1.0783292422355949e-05



In [21]:
# Visualize how the MSE changes with different hyperparameters (3D meshgrid, rotatable plot))
import plotly.graph_objects as go

# Create DataFrame
results_df = pd.DataFrame(results, columns=['n_estimators', 'max_depth', 'mse'])

# Prepare data for 3D plot
X = results_df['n_estimators']
Y = results_df['max_depth']
Z = results_df['mse']

# Create 3D plot
fig = go.Figure(data=[go.Mesh3d(x=X, y=Y, z=Z, color='lightblue', opacity=0.50)], layout=go.Layout(title='MSE vs. Hyperparameters'))

# Color the plot based on mse value
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=12, color=Z, colorscale='Viridis', opacity=0.8)))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='Number of Estimators',
        yaxis_title='Max Depth',
        zaxis_title='Mean Squared Error'
    )
)

# Show plot
fig.show()

#### Part 2.4: Random Forest Conclusion

Based on the above results of Grid Search, we see that the best hyperparameters for the Random Forest model are `n_estimators=400` and `max_depth=30`. The MSE for this model is `8.782553467944984e-06`, which is a good value to shoot for given our output is a percentage. This means that the model is able to predict the readmission rate with an error of 0.0008782553467944984, which is a very small error on the scale of 0-1 that our output is on.

Evaluating why these parameters likely were the sweet-spot for accuracy while balancing the number of trees and branches within those trees, I believe that the model was able to capture the complexity of the data while not overfitting. The model was able to generalize well to the test data and predict the readmission rate with a very small error.


## Part 3: Gradient Boosting Predictions

Beyond `Random Forest`, another great application of machine learning to regression problems is `Gradient Boosting`. If you don't know what Gradient Boosting is, you can read more about it [here](https://en.wikipedia.org/wiki/Gradient_boosting). In summary, Gradient Boosting is a machine learning technique for regression and classification problems that produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.

To summarize the difference between `Gradient Boosting` and `Random Forests`, the main difference is that `Gradient Boosting` builds trees sequentially, where each tree tries to correct the mistakes of the previous one. On the other hand, `Random Forests` builds trees independently, where each tree is built from a sample of the data.

This section will be an evaluation similar to that seen in `Part 2`, allowing comparison of the two approaches to this domain.

In [ ]:
# TODO

# NOTE: If these models are too good, could make a bigger split
# NOTE: When testing, you should have a validation set
# NOTE: Could try the model without doing feature engineering
# NOTE: After gradient boosting, could do a simple neural network (non-deep) to see if we even need the complex deep learning models